# Overview

This notebook contains code and analysis for the simulations of affinity maturation presented in

### Role of framework mutations and antibody flexibility in the evolution of broadly neutralizing antibodies  

Victor Ovchinnikov$^{1,*}$, Joy E. Louveau$^{2,*}$, John P. Barton$^{3-6,*}$, Martin Karplus$^{1,7,\#}$, and Arup K. Chakraborty$^{3-6,8,9,\#}$  

$^1$ Harvard University, Department of Chemistry and Chemical Biology (Cambridge, MA, United States)  
$^2$ Massachusetts Institute of Technology, Harvard-MIT Division of Health Sciences and Technology (Cambridge, MA, United States)  
$^3$ Massachusetts Institute of Technology, Department of Chemical Engineering (Cambridge, MA, United States)  
$^4$ Massachusetts Institute of Technology, Department of Physics (Cambridge, MA, United States)  
$^5$ Massachusetts Institute of Technology, Institute for Medical Engineering and Science (Cambridge, MA, United States)  
$^6$ Ragon Institute of MGH, MIT and Harvard, (Cambridge, MA, United States)  
$^7$ Laboratoire de Chimie Biophysique, ISIS, Universite de Strasbourg, France  
$^8$ Massachusetts Institute of Technology, Department of Biological Engineering (Cambridge, MA, United States)  
$^9$ Massachusetts Institute of Technology, Department of Chemistry (Cambridge, MA, United States)  
$^*$ these authors contributed equally  
$^\#$ correspondence to arupc@mit.edu or marci@tammy.harvard.edu  

# Contents

In addition to this notebook, this repository contains

`am.py` - Python code used for affinity maturation simulations  
`mplot.py` - a Python library for generating plots through matplotlib, see the GitHub repository [here](https://github.com/johnbarton/mplot)  
`/data` - a folder containing output from the affinity maturation simulations and other analysis in this notebook  
`/figures` - a folder containings figures generated in this notebook

This notebook contains the following sections
1. Loading libraries/functions
2. Data analysis and plotting figures
3. Recording of parameters for `am.py` used in alternative simulations

# 1. Load necessary libraries and functions

In [1]:
# Full library list and version numbers

print('This notebook was prepared using:')

import sys
print('python version %s' % sys.version)

import numpy as np
print('numpy version %s' % np.__version__)

import pandas as pd
print('pandas version %s' % pd.__version__)

import matplotlib
print('matplotlib version %s' % matplotlib.__version__)

This notebook was prepared using:
python version 3.6.3 (default, Oct  4 2017, 06:09:15) 
[GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.37)]
numpy version 1.13.3
pandas version 0.21.0
matplotlib version 2.1.0


### Figure settings and libraries

In [2]:
import os
from copy import deepcopy
from importlib import reload

import matplotlib.cm as cm
import matplotlib.pyplot as plot
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg

import mplot as mp

# Plot conventions

def cm2inch(x): return float(x)/2.54
single_column   = cm2inch(8.5)
one_five_column = cm2inch(11.4)
double_column   = cm2inch(17.4)

goldr        = (1.0 + np.sqrt(5)) / 2.0
bkcolor      = '#252525'
altcolor     = '#3182bd'
fontfamily   = 'Arial'
sizeline     = 0.6
sizesublabel = 9
sizelabel    = 8
sizetick     = 7
ticklength   = 3
tickpad      = 3
axwidth      = 0.4

figprops = {
    'transparent' : True,
    'bbox_inches' : 'tight'
    }

def_barprops = {
    'lw'          : sizeline/2,
    'width'       : 0.25,
    'edgecolor'   : bkcolor,
    'align'       : 'center', #other option: edge
    'orientation' : 'vertical'
    }

def_histprops = {
    'histtype'    : 'bar',
    'lw'          : sizeline/2,
    'rwidth'      : 0.8,
    'ls'          : 'solid',
    'edgecolor'   : bkcolor,
    'alpha'       : 0.5
    }

def_lineprops = {
    'lw' : sizeline, 
    'ls' : '-'
    }

def_labelprops = {
    'family' : fontfamily,
    'size'   : sizelabel,
    'color'  : bkcolor
    }

def_sublabelprops = {
    'family' : fontfamily,
    'size'   : sizesublabel,
    'weight' : 'bold',
    'ha'     : 'center',
    'va'     : 'center',
    'color'  : 'k'
    }

def_ticklabelprops = {
    'family' : fontfamily,
    'size'   : sizetick,
    'color'  : bkcolor
    }

def_tickprops = {
    'length'    : ticklength,
    'width'     : axwidth/2,
    'pad'       : tickpad,
    'axis'      : 'both',
    'direction' : 'out',
    'colors'    : bkcolor,
    'bottom'    : True,
    'left'      : True,
    'top'       : False,
    'right'     : False
    }

def_minortickprops = {
    'length'    : ticklength-1.25,
    'width'     : axwidth/2,
    'axis'      : 'both',
    'direction' : 'out',
    'which'     : 'minor',
    'color'     : bkcolor
    }

def_axprops = {
    'linewidth' : axwidth,
    'linestyle' : '-',
    'color'     : bkcolor
    }

params = {'text.usetex': False, 'mathtext.fontset': 'stixsans', 'mathtext.default' : 'regular'}
plot.rcParams.update(params)

### Important functions

In [3]:
def getdE(o=3.0, mu=1.9, sigma=0.5):
    """ Return a random (change in) binding energy following a shifted log-normal distribution. """
    return o - np.exp(np.random.normal(mu, sigma))

def getE(Ec, Ev, Q, lam=0.9):
    """ Return binding energy given raw binding energies with conserved and variable regions, and flexibility. """
    E0 = 4.
    return (Q * ((lam * Ec) + ((1. - lam) * Ev))) + ((1. - Q) * E0)

# 2. Analysis and figures

### Figure S4. Population trajectories of successful and unsusccessful GCs, single antigen test case

In [4]:
# Load the data here -- single antigen case

df    = pd.read_csv(        'data/1ag-output-total.csv.gz', memory_map=True)
dfLC  = pd.read_csv('data/1ag-output-largest-clone.csv.gz', memory_map=True)
dfMem = pd.read_csv(       'data/1ag-output-memory.csv.gz', memory_map=True)

nTrials = 10000
finalE  = []
nMut    = []

for t in range(nTrials):
    dfT = dfMem[dfMem['trial']==t]
    if len(dfT)>0:
        idx = np.argmax(np.array(dfT['number']))
        nMut.append(dfT.iloc[idx].mutations)
        tempEc = dfT.iloc[idx].Ec
        tempEv = dfT.iloc[idx].Ev0
        tempQ  = dfT.iloc[idx].Q
        finalE.append(getE(tempEc, tempEv, tempQ))

In [5]:
# Plot the figure

nTrials = 10000
plotCut = 1000  # number of actual lines to plot
nSurv   = 0
hSurv   = []
hDie    = []

w     = one_five_column
goldh = (1.0*w)/goldr
fig   = plot.figure(figsize=(w, goldh))
gs1   = gridspec.GridSpec(3, 1, width_ratios=[1.00], height_ratios=[1.0,1.0,1.0])
gs1.update(left=0.02, right=0.58, hspace=0.2)
ax1   = plot.subplot(gs1[:2, 0]) # trajectories
ax2   = plot.subplot(gs1[ 2, 0]) # end cycle histogram
gs2   = gridspec.GridSpec(2, 1, width_ratios=[1.00], height_ratios=[1.0,1.0])
gs2.update(left=0.72, right=0.98, bottom=0.3, hspace=0.8)
ax3   = plot.subplot(gs2[ 0, 0]) # kde - mutations
ax4   = plot.subplot(gs2[ 1, 0]) # kde - final energy


# PART 1: Trajectories

print('Plotting GC trajectories...')

legendprops = { 'loc' : 4, 'frameon' : False, 'scatterpoints' : 1, 'handletextpad' : 0.1, 
                'prop' : {'size' : sizelabel}, 'ncol' : 1 }
lineprops   = { 'lw' : sizeline, 'ls' : '-', 'alpha' : 0.2 }
cSurv       = '#FF7F00'
cDie        = bkcolor

pprops = { 'xlim'      : [ 0, 250],
           'xticks'    : [],
           'ylim'      : [ 0, 1536],
           'yticks'    : [ 0,  250, 500, 750, 1000, 1250, 1500],
           'ylabel'    : 'Number of B cells',
           'plotprops' : lineprops,
           'hide'      : ['bottom'],
           'sublabel'  : 'A',
           'sublabelcoords' : [-0.18, 1.00] }

for t in range(nTrials):
    dfT  = df[df['trial']==t]
    x, y = np.array(dfT['cycle']), np.array(dfT['number recycled'])
    if y[-1]>0: 
        nSurv += 1
        hSurv.append(x[-1])
        if t<plotCut: mp.line(ax=ax1, x=[x], y=[y], colors=[cSurv], **pprops)
    else:
        hDie.append(x[-1])
        if t<plotCut: mp.line(ax=ax1, x=[x], y=[y], colors=[cDie],  **pprops)


offPlot    = [[-10,-10],[-10,-10]]
survString = '%d%%' % np.round(float(nSurv)/float(nTrials) * 100) + ' succeed'
dieString  = '%d%%' % np.round(100. - (float(nSurv)/float(nTrials) * 100)) + ' die'
ax1.text(185, 250, survString, ha='left', va='center', color=cSurv, family=fontfamily, size=sizelabel)
ax1.text(185, 110,  dieString, ha='left', va='center', color= cDie, family=fontfamily, size=sizelabel)
mp.plot(type='line', ax=ax1, x=offPlot, y=offPlot, colors=[cSurv,cDie], **pprops)


# PART 2: Histograms

print('Plotting supplementary info...')

histprops                = { 'histtype' : 'bar', 'lw' : sizeline/2, 'rwidth' : 0.8, 'ls' : 'solid', 'alpha' : 0.7,
                             'align' : 'left' }
pprops['hide']           = []
pprops['xlim']           = [ 0, 250]
pprops['xticks']         = [ 0,  50, 100, 150, 200, 250]
pprops['xaxy']           = 1
pprops['xlabel']         = 'Cycles'
pprops['ylim']           = [0, 0.15]
pprops['yticks']         = [0, 0.05, 0.10, 0.15]
pprops['yticklabels']    = ['0', '5', '10', '15']
pprops['ylabel']         = 'Frequency (%)'
pprops['plotprops']      = histprops
pprops['sublabel']       = 'B'
pprops['sublabelcoords'] = [-0.18, 1.00]

mp.plot(type='hist', ax=ax2, x=[hSurv, hDie], colors=[cSurv, cDie], **pprops)


# PART 3: KDEs
cKDE = '#3182bd'

pprops['plotprops']      = histprops
pprops['bins']           = np.arange(0,15+1,1)
pprops['xlim']           = [ 0, 15]
pprops['xticks']         = [ 0, 5, 10, 15]
pprops['xlabel']         = 'Number of mutations'
pprops['ylim']           = [0, 0.30]
pprops['yticks']         = [0, 0.10, 0.20, 0.30]
pprops['yticklabels']    = ['0', '10', '20', '30']
pprops['ylabel']         = 'Frequency (%)'
pprops['sublabel']       = 'C'
pprops['sublabelcoords'] = [-0.40, 1.00]

mp.plot(type='hist', ax=ax3, x=[nMut], colors=[cKDE], **pprops)

del pprops['yticklabels']
pprops['plotprops']      = def_lineprops
pprops['xlim']           = [ 0, 10]
pprops['xticks']         = [ 0, 2.5, 5, 7.5, 10]
pprops['xticklabels']    = [r'$0$', r'$-2.5$', r'$-5$', r'$-7.5$', r'$-10$']
pprops['xlabel']         = 'Binding free energy'
pprops['ylim']           = [0, 0.3]
pprops['yticks']         = [0, 0.1, 0.2, 0.3]
pprops['ylabel']         = 'Density'
pprops['sublabel']       = 'D'
pprops['sublabelcoords'] = [-0.40, 1.00]

mp.plot(type='kde', ax=ax4, x=[finalE], colors=[cKDE], **pprops)


# Save data
plot.savefig('figures/figure-S4-population-trajectories.pdf', **figprops)
plot.close(fig)

print('Done.')

Plotting GC trajectories...
Plotting supplementary info...
Done.


### Figure 6. Accumulation of mutations in the FR and CDR regions

In [6]:
# Load the data here

dfHi = pd.read_csv('data/hiEc-output-largest-clone.csv.gz', memory_map=True)
dfLo = pd.read_csv('data/loEc-output-largest-clone.csv.gz', memory_map=True)
        
dfHi.set_index(['trial'], inplace=True)
dfLo.set_index(['trial'], inplace=True)

genCut = [25, 150, 400]

xHi, yHi = [], []
xLo, yLo = [], []

lastdfHi = dfHi[dfHi['generation']==0]
lastdfLo = dfLo[dfLo['generation']==0]

for t in range(1,501):
    nextdfHi = dfHi[dfHi['generation']==t].combine_first(lastdfHi)
    lastdfHi = nextdfHi
    nextdfLo = dfLo[dfLo['generation']==t].combine_first(lastdfLo)
    lastdfLo = nextdfLo
    
    if t in genCut:
        xHi.append(np.array(nextdfHi['CDR_mutations']))
        yHi.append(np.array(nextdfHi['FR_mutations']))
        xLo.append(np.array(nextdfLo['CDR_mutations']))
        yLo.append(np.array(nextdfLo['FR_mutations']))

In [7]:
# Plot the figure

w     = double_column
goldh = (2.0/3.)*w
fig   = plot.figure(figsize=(w, goldh))

HiBox = [0.20, 0.98, 0.60, 0.90]
LoBox = [0.20, 0.98, 0.16, 0.46]

gsS   = gridspec.GridSpec(1, 1, width_ratios=[1.0], height_ratios=[1.0])
gsS.update(left=HiBox[0], right=HiBox[1], bottom=0.93, top=1.0)
axS   = plot.subplot(gsS[ 0, 0])

gsHi  = gridspec.GridSpec(1, 3, width_ratios=[1.0, 1.0, 1.0])
gsHi.update(left=HiBox[0], right=HiBox[1], bottom=HiBox[2], top=HiBox[3], hspace=0.5, wspace=0.5)
gsLo  = gridspec.GridSpec(1, 3, width_ratios=[1.0, 1.0, 1.0])
gsLo.update(left=LoBox[0], right=LoBox[1], bottom=LoBox[2], top=LoBox[3], hspace=0.5, wspace=0.5)

axHi  = [plot.subplot(gsHi[i]) for i in range(len(genCut))]
axLo  = [plot.subplot(gsLo[i]) for i in range(len(genCut))]

# Overhead line

lineprops = { 'lw' : axwidth/2., 'ls' : '-', 'alpha' : 1.0 }
pprops    = { 'xlim' : [0, 1], 'xticks' : [], 'ylim' : [0, 1], 'yticks' : [], 
              'hide' : ['top','bottom','left','right'], 'plotprops' : lineprops }
tprops    = dict(color=bkcolor, family=fontfamily, size=sizelabel)
axS.text(3./6.-0.005, 1.0, 'Time (generations)', ha='center', va='top',    **tprops)
axS.text(1./6.-0.045, 0.3,              r'$25$', ha='center', va='center', **tprops)
axS.text(3./6.-0.005, 0.3,             r'$150$', ha='center', va='center', **tprops)
axS.text(5./6.+0.055, 0.3,             r'$400$', ha='center', va='center', **tprops)
ay = 0.05
axS.arrow(x=0, y=ay, dx=1, dy=0, head_width=0.25, head_length=0.025, overhang=0.5, fc=bkcolor,
             ec=None, lw=sizeline/2., clip_on=False, zorder=100)
mp.plot(type='line', ax=axS, x=[[0,1]], y=[[ay,ay]], colors=[bkcolor], **pprops)

# Number of CDR/FWR mutations

print('Plotting number of mutations...')

levels = [0.02] + list(0.02 * np.logspace(1., 5., num=5, base=1.5))

legendprops  = { 'loc' : 4, 'frameon' : False, 'scatterpoints' : 1, 'handletextpad' : 0.1, 
                 'prop' : {'size' : sizelabel}, 'ncol' : 1 }
contourprops = { 'cmap' : 'Greens', 'colors' : bkcolor, 'extend' : 'max', 'linewidths' : sizeline/2.,
                 'vmin' : 0.001, 'vmax' : 0.15 , 'levels' : levels }

dx     = 0.1
dy     = dx
cx     = np.arange(0., 12.+dx, dx)
cy     = np.arange(0., 12.+dy, dy)
xx, yy = np.meshgrid(cx, cy)

sx = 0.9
sy = sx

pprops = { 'xlim'      : [ 0., 12.],
           'xticks'    : [ 0., 4., 8., 12.],
           'ylim'      : [ 0., 12.],
           'yticks'    : [ 0., 4., 8., 12.],
           'xlabel'    : 'CDR mutations',
           'ylabel'    : 'Framework mutations',
           'plotprops' : contourprops,
           'zorder'    : 10 }

for i in range(len(xHi)):
    zz = np.zeros(xx.shape)
    for k in range(len(xHi[i])):
        zz += np.exp(-((xHi[i][k]-xx)**2/sx**2) - ((yHi[i][k]-yy)**2/sy**2))
    zz /= float(len(xHi[i]))
    mp.plot(type='contour', ax=axHi[i], x=[xx], y=[yy], z=[zz], **pprops)

for i in range(len(xLo)):
    zz = np.zeros(xx.shape)
    for k in range(len(xLo[i])):
        zz += np.exp(-((xLo[i][k]-xx)**2/sx**2) - ((yLo[i][k]-yy)**2/sy**2))
    zz /= float(len(xLo[i]))
    mp.plot(type='contour', ax=axLo[i], x=[xx], y=[yy], z=[zz], **pprops)

# Background

axS.text(HiBox[0]-0.06, HiBox[3]-0.01, 'A', transform=fig.transFigure, **def_sublabelprops)
axS.text(LoBox[0]-0.06, LoBox[3]-0.01, 'B', transform=fig.transFigure, **def_sublabelprops)

cBG   = '#F5F5F5'
dx    = 0.08
dy    = 0.09
bgHi  = axS.axis()
recHi = matplotlib.patches.Rectangle(xy=(HiBox[0]-dx,HiBox[2]-dy), width=0.8+dx, height=HiBox[3]-HiBox[2]+(1.2*dy), 
                                     transform=fig.transFigure, ec=None, fc=cBG, clip_on=False, zorder=-100)
recHi = axS.add_patch(recHi)

bgLo  = axS.axis()
recLo = matplotlib.patches.Rectangle(xy=(LoBox[0]-dx,LoBox[2]-dy), width=0.8+dx, height=LoBox[3]-LoBox[2]+(1.2*dy), 
                                     transform=fig.transFigure, ec=None, fc=cBG, clip_on=False, zorder=-100)
recLo = axS.add_patch(recLo)
    
# Save data
plot.savefig('figures/figure-6-mutations.pdf', dpi = 1000, 
              facecolor = fig.get_facecolor(), edgecolor=None, **figprops)
plot.close(fig)

print('Done.')

Plotting number of mutations...
Done.


### Analysis for Figures 4 and S5. Breadth comparisons at different binding energy cutoffs

In [8]:
# Create alternate "test panel" of 100 sequences with different Ev

nTrials   = 10000
nPanel    = 100
testPanel = np.array([getdE(o=0, mu=3.0, sigma=1.0) for i in range(nPanel)])

# Read through largest clone histories and output binding energies with the test panel

fIn  = ['data/hiEC-output-largest-clone.csv.gz',
        'data/loEC-output-largest-clone.csv.gz',
        'data/hiEC-l0.8-output-largest-clone.csv.gz',
        'data/loEC-l0.8-output-largest-clone.csv.gz']
fOut = ['data/hiEc-panel-breadth.csv.gz', 
        'data/loEc-panel-breadth.csv.gz',
        'data/hiEc-l0.8-panel-breadth.csv.gz', 
        'data/loEc-l0.8-panel-breadth.csv.gz']

for k in range(len(fIn)):
    if os.path.isfile(fOut[k]):
        continue
    else:
        f  = open(fOut[k], 'w')
        f.write('trial,generation,Q,Ec%s\n' % (''.join([',Ev%d' % (i+1) for i in range(nPanel)])))

        df = pd.read_csv(fIn[k], memory_map=True)

        for t in range(nTrials):
            trialDf = df[df.trial==t]

            if len(trialDf)==0: continue

            # Set starting binding energies with test panel

            panelEc = trialDf.iloc[0].Ec
            panelEv = deepcopy(testPanel)

            f.write('%d,%d,%lf,%lf' % (trialDf.iloc[0].trial,trialDf.iloc[0].generation,trialDf.iloc[0].Q,panelEc))
            for i in range(len(testPanel)): f.write(',%lf' % panelEv[i])
            f.write('\n')

            # Iterate over successive mutations and update binding energies

            for k in range(1,len(trialDf)):

                if trialDf.iloc[k].Q==trialDf.iloc[k-1].Q:
                    dEc = trialDf.iloc[k].Ec-trialDf.iloc[k-1].Ec
                    if dEc==0:
                        for i in range(len(testPanel)): panelEv[i] += getdE()
                    else: panelEc += dEc

                f.write('%d,%d,%lf,%lf' % (trialDf.iloc[k].trial,trialDf.iloc[k].generation,trialDf.iloc[k].Q,panelEc))
                for i in range(len(testPanel)): f.write(',%lf' % panelEv[i])
                f.write('\n')
                f.flush()

        f.close() # file should be gzipped externally

### Figure 4. Trajectories of binding energy/flexibility during affinity maturation

In [9]:
# Load data: Q vs Ec plot

dfHi = pd.read_csv('data/hiEc-output-largest-clone.csv.gz', memory_map=True)
dfLo = pd.read_csv('data/loEc-output-largest-clone.csv.gz', memory_map=True)

dfHi.set_index(['trial'], inplace=True)
dfLo.set_index(['trial'], inplace=True)

genCut = [25, 150, 400]

xHi, yHi = [], []
xLo, yLo = [], []

lastdfHi = dfHi[dfHi['generation']==0]
lastdfLo = dfLo[dfLo['generation']==0]

for t in range(1,501):
    nextdfHi = dfHi[dfHi['generation']==t].combine_first(lastdfHi)
    lastdfHi = nextdfHi
    nextdfLo = dfLo[dfLo['generation']==t].combine_first(lastdfLo)
    lastdfLo = nextdfLo
    
    if t in genCut:
        xHi.append(np.array(nextdfHi['Ec']))
        yHi.append(np.array(nextdfHi['Q']))
        xLo.append(np.array(nextdfLo['Ec']))
        yLo.append(np.array(nextdfLo['Q']))
        
# Load data: breadth plot

ECut = [3.0, 6.0, 9.0]

dfHi = pd.read_csv('data/hiEc-panel-breadth.csv.gz', memory_map=True)
dfLo = pd.read_csv('data/loEc-panel-breadth.csv.gz', memory_map=True)

dfHi.set_index(['trial'], inplace=True)
dfLo.set_index(['trial'], inplace=True)

nPanel     = 100
bHi        = []
bLo        = []
bHiTrueMed = []
bLoTrueMed = []
EvLabel    = ['Ev%d' % (i+1) for i in range(nPanel)]
denom      = float(nPanel)

lastdfHi = dfHi[dfHi['generation']==0]
lastdfLo = dfLo[dfLo['generation']==0]

for t in range(1,501):
    nextdfHi = dfHi[dfHi['generation']==t].combine_first(lastdfHi)
    lastdfHi = nextdfHi
    nextdfLo = dfLo[dfLo['generation']==t].combine_first(lastdfLo)
    lastdfLo = nextdfLo
    
    if t in genCut:
        tempQ  = np.array([ q * np.ones(nPanel) for  q in nextdfHi.Q])
        tempEc = np.array([ec * np.ones(nPanel) for ec in nextdfHi.Ec])
        EHi    = np.array(getE(tempEc, np.array(nextdfHi[EvLabel]), tempQ))
        
        tempQ  = np.array([ q * np.ones(nPanel) for  q in nextdfLo.Q])
        tempEc = np.array([ec * np.ones(nPanel) for ec in nextdfLo.Ec])
        ELo    = np.array(getE(tempEc, np.array(nextdfLo[EvLabel]), tempQ))
        
        bHi.append([[np.sum(E>c)/denom for E in EHi] for c in ECut])
        bLo.append([[np.sum(E>c)/denom for E in ELo] for c in ECut])
        
        bHiTrueMed.append([np.median(E) for E in EHi])
        bLoTrueMed.append([np.median(E) for E in ELo])

In [10]:
# Plot the figure

nTrials = 10000
reload(mp)

w     = double_column
goldh = (3.3/3.0)*w
fig   = plot.figure(figsize=(w, goldh))

HiBox = [0.20, 0.98, 0.61, 0.91]
LoBox = [0.20, 0.98, 0.23, 0.53]
LLBox = [0.10, 0.30, 0.00, 0.16]

gsS   = gridspec.GridSpec(1, 2, width_ratios=[1.0, 2.5], height_ratios=[1.0])
gsS.update(left=0, right=HiBox[1], bottom=0.93, top=1.0)
gsHiS = gridspec.GridSpec(1, 1, width_ratios=[1.0], height_ratios=[1.0])
gsHiS.update(left=0.02, right=0.12, bottom=HiBox[2]+((HiBox[3]-HiBox[2])*0.5/1.3), top=HiBox[3])
gsLoS = gridspec.GridSpec(1, 1, width_ratios=[1.0], height_ratios=[1.0])
gsLoS.update(left=0.02, right=0.12, bottom=LoBox[2]+((LoBox[3]-LoBox[2])*0.5/1.3), top=LoBox[3])
axS   = [plot.subplot(gsS[ 0, 1]), plot.subplot(gsHiS[ 0, 0]), plot.subplot(gsLoS[ 0, 0])]
axKey = plot.subplot(gsS[0, 0])

gsHi  = gridspec.GridSpec(2, 3, height_ratios=[1.0, 0.55], width_ratios=[1.0, 1.0, 1.0])
gsHi.update(left=HiBox[0], right=HiBox[1], bottom=HiBox[2], top=HiBox[3], hspace=0.2, wspace=0.2)
gsLo  = gridspec.GridSpec(2, 3, height_ratios=[1.0, 0.55], width_ratios=[1.0, 1.0, 1.0])
gsLo.update(left=LoBox[0], right=LoBox[1], bottom=LoBox[2], top=LoBox[3], hspace=0.2, wspace=0.2)
axHi  = [[plot.subplot(gsHi[0, i]), plot.subplot(gsHi[1, i])] for i in range(len(genCut))]
axLo  = [[plot.subplot(gsLo[0, i]), plot.subplot(gsLo[1, i])] for i in range(len(genCut))]

gsLL = gridspec.GridSpec(1, 1, height_ratios=[1.0], width_ratios=[1.0])
gsLL.update(left=LLBox[0], right=LLBox[1], bottom=LLBox[2], top=LLBox[3], hspace=0.4, wspace=0.1)
axLL = [plot.subplot(gsLL[0, 0])]

# Overhead line

lineprops = { 'lw' : axwidth/2., 'ls' : '-', 'alpha' : 1.0 }
pprops    = { 'xlim' : [0, 1], 'xticks' : [], 'ylim' : [0, 1], 'yticks' : [], 
              'hide' : ['top','bottom','left','right'], 'plotprops' : lineprops }
tprops    = dict(color=bkcolor, family=fontfamily, size=sizelabel)
axS[0].text(2.4/6.-0.005, 0.6, 'Time (generations)', ha='center', va='top',    **tprops)
axS[0].text(           0, 0.2,              r'$25$', ha='center', va='center', **tprops)
axS[0].text(2.4/6.-0.005, 0.2,             r'$150$', ha='center', va='center', **tprops)
axS[0].text(4.9/6.+0.015, 0.2,             r'$400$', ha='center', va='center', **tprops)
ay = 0.05
axS[0].arrow(x=-0.30, y=ay, dx=1+0.30, dy=0, head_width=0.25, head_length=0.025, overhang=0.5, fc=bkcolor,
             ec=None, lw=sizeline/2., clip_on=False, zorder=100)
mp.plot(type='line', ax=axS[0], x=[[0,1]], y=[[ay,ay]], colors=[bkcolor], **pprops)

# PART 1: densities

print('Plotting densities...')

legendprops  = { 'loc' : 4, 'frameon' : False, 'scatterpoints' : 1, 'handletextpad' : 0.1, 
                 'prop' : {'size' : sizelabel}, 'ncol' : 1 }
contourprops = { 'cmap' : 'Oranges', 'colors' : bkcolor, 'vmin' : 0.001, 'vmax' : 0.10, 'extend' : 'max',
                 'linewidths' : sizeline/2., 'levels' : [0.001, 0.005, 0.015, 0.035, 0.075, 0.15]}

dx     = 0.1
dy     = dx * (1. / 16.)
cx     = np.arange(-2., 14.+dx, dx)
cy     = np.arange( 0.,  1.+dy, dy)
xx, yy = np.meshgrid(cx, cy)

sx = 0.5
sy = sx * (1. / 16.)
    
pprops = { 'xlim'      : [-2., 14.],
           'xticks'    : [0.,  4.,  8., 12.],
           'xticklabels' : [r'$0$', r'$-4$', r'$-8$', r'$-12$'],
           'xminorticks' : [-2., 2., 6., 10., 14.],
           'ylim'      : [ 0., 1.],
           'yticks'    : [ 0., 0.25, 0.50, 0.75, 1.00],
           'xlabel'    : r'$E_c$',
           'ylabel'    : r'$Q$',
           'plotprops' : contourprops,
           'zorder'    : 10 }

r      = 1.
cprops = dict(width=r, height=r/14., ec='k', fc='w', alpha=1.0, zorder=100)

for i in range(len(genCut)):
    tempprops = pprops.copy()
    if i>0:
        del tempprops['ylabel']
        tempprops['yticks']   = []
        tempprops['hide']     = ['left']
    zz = np.zeros(xx.shape)
    for k in range(len(xHi[i])):
        zz += np.exp(-((xHi[i][k]-xx)**2/sx**2) - ((yHi[i][k]-yy)**2/sy**2))
    zz /= float(len(xHi[i]))
    mp.plot(type='contour', ax=axHi[i][0], x=[xx], y=[yy], z=[zz], **tempprops)
    circHi = matplotlib.patches.Ellipse(xy=(4.,0.8), **cprops)
    axHi[i][0].add_patch(circHi)
    if i==0:
        axHi[i][0].annotate('Initial\nparameters', xy=(3.7, 0.78), xytext=(1.75, 0.50), ha='center', va='center',
                            arrowprops=dict(arrowstyle='simple', fc='k', mutation_scale=10., shrinkA=3., shrinkB=3.), 
                            **def_labelprops)
    if i==len(genCut)-1:
        axHi[i][0].text(2., 0.90, 'CH103-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})

for i in range(len(genCut)):
    tempprops = pprops.copy()
    if i>0:
        del tempprops['ylabel']
        tempprops['yticks']   = []
        tempprops['hide']     = ['left']
    zz = np.zeros(xx.shape)
    for k in range(len(xLo[i])):
        zz += np.exp(-((xLo[i][k]-xx)**2/sx**2) - ((yLo[i][k]-yy)**2/sy**2))
    zz /= float(len(xLo[i]))
    mp.plot(type='contour', ax=axLo[i][0], x=[xx], y=[yy], z=[zz], **tempprops)
    circLo = matplotlib.patches.Ellipse(xy=(0.,0.8), **cprops)
    axLo[i][0].add_patch(circLo)
    if i==len(genCut)-1:
        axLo[i][0].text(-0.5, 0.90, 'PGT121-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})
        axLo[i][0].text(-2., 0.35, '3BNC60-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})

# PART 2: breadth

print('Plotting breadth...')

legendprops = { 'loc' : 4, 'frameon' : False, 'scatterpoints' : 1, 'handletextpad' : 0.1, 
                'prop' : {'size' : sizelabel}, 'ncol' : 1 }
lineprops   = { 'lw' : sizeline, 'ls' : '-', 'alpha' : 1.0 }
cHist       = '#3182bd'

histprops = { 'histtype' : 'bar', 'lw' : sizeline/2, 'rwidth' : 0.8, 'ls' : 'solid', 'alpha' : 0.7, 'ec' : bkcolor }

pprops = { 'xlim'        : [-2., 10.],
           'xticks'      : [0.,  4., 8.],
           'xticklabels' : [r'$0$', r'$-4$', r'$-8$'],
           'xminorticks' : [-2,  2., 6., 10.],
           'ylim'        : [ 0., 0.50],
           'yticks'      : [ 0., 0.25, 0.50],
           'yticklabels' : [  0,   25,   50],
           'xlabel'      : 'Median binding free energy',
           'ylabel'      : 'Frequency (%)',
           'plotprops'   : histprops,
           'bins'        : np.arange(-2.0, 10.5, 0.5),
           'combine'     : True }

for g in range(len(genCut)):
    tempprops = pprops.copy()
    if g>0: 
        del tempprops['ylabel']
        tempprops['yticks'] = []
        tempprops['hide']   = ['left']
    mp.plot(type='hist', ax=axHi[g][1], x=[bHiTrueMed[g]], colors=[cHist], **tempprops)
    mp.plot(type='hist', ax=axLo[g][1], x=[bLoTrueMed[g]], colors=[cHist], **tempprops)
    
# PART 3: Q vs breadth

print('Plotting Q vs breadth...')

dx     = 0.05
dy     = dx * (1. / 7.)
cx     = np.arange(1., 8.+dx, dx)
cy     = np.arange(0., 1.+dy, dy)
xx, yy = np.meshgrid(cx, cy)

sx = 0.4
sy = sx * (1. / 7.)

zz = np.zeros(xx.shape)
for k in range(len(bLoTrueMed[-1])):
    zz += np.exp(-((bLoTrueMed[-1][k]-xx)**2/sx**2) - ((yLo[-1][k]-yy)**2/sy**2))
zz /= float(len(bLoTrueMed[-1]))

contourprops['cmap'] = 'Blues'
scatterprops = { 'lw' : 0, 's' : 8., 'marker' : 'o' }

pprops = { 'xlim'      : [ 1., 8.],
           'xticks'    : [ 2., 4., 6., 8.],
           'xticklabels' : [r'$-2$', r'$-4$', r'$-6$', r'$-8$'],
           'xaxstart'  : 1,
           'ylim'      : [ 0., 1.],
           'yticks'    : [ 0., 0.25, 0.50, 0.75, 1.00],
           'xlabel'    : 'Median binding free energy',
           'ylabel'    : r'$Q$',
           'plotprops' : contourprops,
           'zorder'    : 10 }

mp.plot(type='contour', ax=axLL[0], x=[xx], y=[yy], z=[zz], **pprops)
axLL[0].text(6.5, 0.80, 'PGT121-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})
axLL[0].text(5.5, 0.25, '3BNC60-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})
    
# Schematic

hiSchem  = mpimg.imread('figures/hiEc.png')
loSchem  = mpimg.imread('figures/loEc.png')
keySchem = mpimg.imread('figures/legend2.png')
axS[1].imshow(hiSchem)
axS[2].imshow(loSchem)
axKey.imshow(keySchem, zorder=-200)
for ax in [axS[1], axS[2], axKey]:
    for a in ['left', 'right', 'bottom', 'top']: 
        ax.spines[a].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
axS[2].text( 0.02, LoBox[3]-0.01, 'B', transform=fig.transFigure, **def_sublabelprops)
axS[1].text( 0.02, HiBox[3]-0.01, 'A', transform=fig.transFigure, **def_sublabelprops)
axLL[0].text(0.02, LoBox[2]-0.08, 'C', transform=fig.transFigure, **def_sublabelprops)
axS[1].text(0.07, (HiBox[2]+HiBox[3])/2.-0.03, 'Strong initial ' + r'$E_c$', ha='center', va='center',
            transform=fig.transFigure, **def_labelprops)
axS[2].text(0.07, (LoBox[2]+LoBox[3])/2.-0.03, 'Weak initial ' + r'$E_c$', ha='center', va='center', 
            transform=fig.transFigure, **def_labelprops)
tprops['size'] = sizelabel
axKey.text(0.080, 0.965,        'viral\nspike', transform=fig.transFigure, ha='center', va='center', **tprops)
axKey.text(0.175, 0.965,   'conserved\nregion', transform=fig.transFigure, ha='center', va='center', **tprops)
axKey.text(0.290, 0.965, 'antibody\nfootprint', transform=fig.transFigure, ha='center', va='center', **tprops)

# Background

cBG   = '#F5F5F5'
dy    = 0.07
bgHi  = axS[1].axis()
recHi = matplotlib.patches.Rectangle(xy=(0,HiBox[2]-0.8*dy), width=1., height=HiBox[3]-HiBox[2]+(1.*dy), 
                                     transform=fig.transFigure, ec=None, fc=cBG, clip_on=False, zorder=-100)
recHi = axS[1].add_patch(recHi)

bgLo  = axS[2].axis()
recLo = matplotlib.patches.Rectangle(xy=(0,LoBox[2]-0.8*dy), width=1., height=LoBox[3]-LoBox[2]+(1.*dy), 
                                     transform=fig.transFigure, ec=None, fc=cBG, clip_on=False, zorder=-100)
recLo = axS[2].add_patch(recLo)
    
# Save data
plot.savefig('figures/figure-4-contour-breadth-Ec-Q.pdf', dpi = 1000, 
              facecolor = fig.get_facecolor(), edgecolor=None, **figprops)
plot.close(fig)

print('Done.')

Plotting densities...
Plotting breadth...
Plotting Q vs breadth...
Done.


### Figure S5. Trajectories of binding energy/flexibility during affinity maturation, smaller overlap ($\lambda = 0.8$)

Note: This figure is the same as Figure 4, just using different input files.

In [10]:
# Load data: Q vs Ec plot

dfHi = pd.read_csv('data/hiEc-l0.8-output-largest-clone.csv.gz', memory_map=True)
dfLo = pd.read_csv('data/loEc-l0.8-output-largest-clone.csv.gz', memory_map=True)

dfHi.set_index(['trial'], inplace=True)
dfLo.set_index(['trial'], inplace=True)

genCut = [25, 150, 400]

xHi, yHi = [], []
xLo, yLo = [], []

lastdfHi = dfHi[dfHi['generation']==0]
lastdfLo = dfLo[dfLo['generation']==0]

for t in range(1,501):
    nextdfHi = dfHi[dfHi['generation']==t].combine_first(lastdfHi)
    lastdfHi = nextdfHi
    nextdfLo = dfLo[dfLo['generation']==t].combine_first(lastdfLo)
    lastdfLo = nextdfLo
    
    if t in genCut:
        xHi.append(np.array(nextdfHi['Ec']))
        yHi.append(np.array(nextdfHi['Q']))
        xLo.append(np.array(nextdfLo['Ec']))
        yLo.append(np.array(nextdfLo['Q']))
        
# Load data: breadth plot

ECut = [3.0, 6.0, 9.0]

dfHi = pd.read_csv('data/hiEc-l0.8-panel-breadth.csv.gz', memory_map=True)
dfLo = pd.read_csv('data/loEc-l0.8-panel-breadth.csv.gz', memory_map=True)

dfHi.set_index(['trial'], inplace=True)
dfLo.set_index(['trial'], inplace=True)

nPanel     = 100
bHi        = []
bLo        = []
bHiTrueMed = []
bLoTrueMed = []
EvLabel    = ['Ev%d' % (i+1) for i in range(nPanel)]
denom      = float(nPanel)

lastdfHi = dfHi[dfHi['generation']==0]
lastdfLo = dfLo[dfLo['generation']==0]

for t in range(1,501):
    nextdfHi = dfHi[dfHi['generation']==t].combine_first(lastdfHi)
    lastdfHi = nextdfHi
    nextdfLo = dfLo[dfLo['generation']==t].combine_first(lastdfLo)
    lastdfLo = nextdfLo
    
    if t in genCut:
        tempQ  = np.array([ q * np.ones(nPanel) for  q in nextdfHi.Q])
        tempEc = np.array([ec * np.ones(nPanel) for ec in nextdfHi.Ec])
        EHi    = np.array(getE(tempEc, np.array(nextdfHi[EvLabel]), tempQ, lam=0.8))
         
        tempQ  = np.array([ q * np.ones(nPanel) for  q in nextdfLo.Q])
        tempEc = np.array([ec * np.ones(nPanel) for ec in nextdfLo.Ec])
        ELo    = np.array(getE(tempEc, np.array(nextdfLo[EvLabel]), tempQ, lam=0.8))
        
        bHi.append([[np.sum(E>c)/denom for E in EHi] for c in ECut])
        bLo.append([[np.sum(E>c)/denom for E in ELo] for c in ECut])
        
        bHiTrueMed.append([np.median(E) for E in EHi])
        bLoTrueMed.append([np.median(E) for E in ELo])

In [11]:
# Plot the figure

nTrials = 10000
reload(mp)

w     = double_column
goldh = (3.3/3.0)*w
fig   = plot.figure(figsize=(w, goldh))

HiBox = [0.20, 0.98, 0.61, 0.91]
LoBox = [0.20, 0.98, 0.23, 0.53]
LLBox = [0.10, 0.30, 0.00, 0.16]

gsS   = gridspec.GridSpec(1, 2, width_ratios=[1.0, 2.5], height_ratios=[1.0])
gsS.update(left=0, right=HiBox[1], bottom=0.93, top=1.0)
gsHiS = gridspec.GridSpec(1, 1, width_ratios=[1.0], height_ratios=[1.0])
gsHiS.update(left=0.02, right=0.12, bottom=HiBox[2]+((HiBox[3]-HiBox[2])*0.5/1.3), top=HiBox[3])
gsLoS = gridspec.GridSpec(1, 1, width_ratios=[1.0], height_ratios=[1.0])
gsLoS.update(left=0.02, right=0.12, bottom=LoBox[2]+((LoBox[3]-LoBox[2])*0.5/1.3), top=LoBox[3])
axS   = [plot.subplot(gsS[ 0, 1]), plot.subplot(gsHiS[ 0, 0]), plot.subplot(gsLoS[ 0, 0])]
axKey = plot.subplot(gsS[0, 0])

gsHi  = gridspec.GridSpec(2, 3, height_ratios=[1.0, 0.55], width_ratios=[1.0, 1.0, 1.0])
gsHi.update(left=HiBox[0], right=HiBox[1], bottom=HiBox[2], top=HiBox[3], hspace=0.2, wspace=0.2)
gsLo  = gridspec.GridSpec(2, 3, height_ratios=[1.0, 0.55], width_ratios=[1.0, 1.0, 1.0])
gsLo.update(left=LoBox[0], right=LoBox[1], bottom=LoBox[2], top=LoBox[3], hspace=0.2, wspace=0.2)
axHi  = [[plot.subplot(gsHi[0, i]), plot.subplot(gsHi[1, i])] for i in range(len(genCut))]
axLo  = [[plot.subplot(gsLo[0, i]), plot.subplot(gsLo[1, i])] for i in range(len(genCut))]

gsLL = gridspec.GridSpec(1, 1, height_ratios=[1.0], width_ratios=[1.0])
gsLL.update(left=LLBox[0], right=LLBox[1], bottom=LLBox[2], top=LLBox[3], hspace=0.4, wspace=0.1)
axLL = [plot.subplot(gsLL[0, 0])]

# Overhead line

lineprops = { 'lw' : axwidth/2., 'ls' : '-', 'alpha' : 1.0 }
pprops    = { 'xlim' : [0, 1], 'xticks' : [], 'ylim' : [0, 1], 'yticks' : [], 
              'hide' : ['top','bottom','left','right'], 'plotprops' : lineprops }
tprops    = dict(color=bkcolor, family=fontfamily, size=sizelabel)
axS[0].text(2.4/6.-0.005, 0.6, 'Time (generations)', ha='center', va='top',    **tprops)
axS[0].text(           0, 0.2,              r'$25$', ha='center', va='center', **tprops)
axS[0].text(2.4/6.-0.005, 0.2,             r'$150$', ha='center', va='center', **tprops)
axS[0].text(4.9/6.+0.015, 0.2,             r'$400$', ha='center', va='center', **tprops)
ay = 0.05
axS[0].arrow(x=-0.30, y=ay, dx=1+0.30, dy=0, head_width=0.25, head_length=0.025, overhang=0.5, fc=bkcolor,
             ec=None, lw=sizeline/2., clip_on=False, zorder=100)
mp.plot(type='line', ax=axS[0], x=[[0,1]], y=[[ay,ay]], colors=[bkcolor], **pprops)

# PART 1: densities

print('Plotting densities...')

legendprops  = { 'loc' : 4, 'frameon' : False, 'scatterpoints' : 1, 'handletextpad' : 0.1, 
                 'prop' : {'size' : sizelabel}, 'ncol' : 1 }
contourprops = { 'cmap' : 'Oranges', 'colors' : bkcolor, 'vmin' : 0.001, 'vmax' : 0.10, 'extend' : 'max',
                 'linewidths' : sizeline/2., 'levels' : [0.001, 0.005, 0.015, 0.035, 0.075, 0.15]}

dx     = 0.1
dy     = dx * (1. / 16.)
cx     = np.arange(-2., 14.+dx, dx)
cy     = np.arange( 0.,  1.+dy, dy)
xx, yy = np.meshgrid(cx, cy)

sx = 0.5
sy = sx * (1. / 16.)
    
pprops = { 'xlim'      : [-2., 14.],
           'xticks'    : [0.,  4.,  8., 12.],
           'xticklabels' : [r'$0$', r'$-4$', r'$-8$', r'$-12$'],
           'xminorticks' : [-2., 2., 6., 10., 14.],
           'ylim'      : [ 0., 1.],
           'yticks'    : [ 0., 0.25, 0.50, 0.75, 1.00],
           'xlabel'    : r'$E_c$',
           'ylabel'    : r'$Q$',
           'plotprops' : contourprops,
           'zorder'    : 10 }

r      = 1.
cprops = dict(width=r, height=r/14., ec='k', fc='w', alpha=1.0, zorder=100)

for i in range(len(genCut)):
    tempprops = pprops.copy()
    if i>0:
        del tempprops['ylabel']
        tempprops['yticks']   = []
        tempprops['hide']     = ['left']
    zz = np.zeros(xx.shape)
    for k in range(len(xHi[i])):
        zz += np.exp(-((xHi[i][k]-xx)**2/sx**2) - ((yHi[i][k]-yy)**2/sy**2))
    zz /= float(len(xHi[i]))
    mp.plot(type='contour', ax=axHi[i][0], x=[xx], y=[yy], z=[zz], **tempprops)
    circHi = matplotlib.patches.Ellipse(xy=(4.,0.8), **cprops)
    axHi[i][0].add_patch(circHi)
    if i==0:
        axHi[i][0].annotate('Initial\nparameters', xy=(3.7, 0.78), xytext=(1.75, 0.50), ha='center', va='center',
                            arrowprops=dict(arrowstyle='simple', fc='k', mutation_scale=10., shrinkA=3., shrinkB=3.), 
                            **def_labelprops)
    if i==len(genCut)-1:
        axHi[i][0].text(2., 0.90, 'CH103-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})

for i in range(len(genCut)):
    tempprops = pprops.copy()
    if i>0:
        del tempprops['ylabel']
        tempprops['yticks']   = []
        tempprops['hide']     = ['left']
    zz = np.zeros(xx.shape)
    for k in range(len(xLo[i])):
        zz += np.exp(-((xLo[i][k]-xx)**2/sx**2) - ((yLo[i][k]-yy)**2/sy**2))
    zz /= float(len(xLo[i]))
    mp.plot(type='contour', ax=axLo[i][0], x=[xx], y=[yy], z=[zz], **tempprops)
    circLo = matplotlib.patches.Ellipse(xy=(0.,0.8), **cprops)
    axLo[i][0].add_patch(circLo)
    if i==len(genCut)-1:
        axLo[i][0].text(-0.5, 0.90, 'PGT121-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})
        axLo[i][0].text(-2., 0.35, '3BNC60-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})

# PART 2: breadth

print('Plotting breadth...')

legendprops = { 'loc' : 4, 'frameon' : False, 'scatterpoints' : 1, 'handletextpad' : 0.1, 
                'prop' : {'size' : sizelabel}, 'ncol' : 1 }
lineprops   = { 'lw' : sizeline, 'ls' : '-', 'alpha' : 1.0 }
cHist       = '#3182bd'

histprops = { 'histtype' : 'bar', 'lw' : sizeline/2, 'rwidth' : 0.8, 'ls' : 'solid', 'alpha' : 0.7, 'ec' : bkcolor }

pprops = { 'xlim'        : [-4., 8.],
           'xticks'      : [-4, 0.,  4., 8.],
           'xticklabels' : [r'$4$', r'$0$', r'$-4$', r'$-8$'],
           'xminorticks' : [-2,  2., 6.],
           'ylim'        : [ 0., 0.50],
           'yticks'      : [ 0., 0.25, 0.50],
           'yticklabels' : [  0,   25,   50],
           'xlabel'      : 'Median binding free energy',
           'ylabel'      : 'Frequency (%)',
           'plotprops'   : histprops,
           'bins'        : np.arange(-4.0, 8.5, 0.5),
           'combine'     : True }

for g in range(len(genCut)):
    tempprops = pprops.copy()
    if g>0: 
        del tempprops['ylabel']
        tempprops['yticks'] = []
        tempprops['hide']   = ['left']
    mp.plot(type='hist', ax=axHi[g][1], x=[bHiTrueMed[g]], colors=[cHist], **tempprops)
    mp.plot(type='hist', ax=axLo[g][1], x=[bLoTrueMed[g]], colors=[cHist], **tempprops)
    
# PART 3: Q vs breadth

print('Plotting Q vs breadth...')

dx     = 0.05
dy     = dx * (1. / 7.)
cx     = np.arange(1., 8.+dx, dx)
cy     = np.arange(0., 1.+dy, dy)
xx, yy = np.meshgrid(cx, cy)

sx = 0.4
sy = sx * (1. / 7.)

zz = np.zeros(xx.shape)
for k in range(len(bLoTrueMed[-1])):
    zz += np.exp(-((bLoTrueMed[-1][k]-xx)**2/sx**2) - ((yLo[-1][k]-yy)**2/sy**2))
zz /= float(len(bLoTrueMed[-1]))

contourprops['cmap'] = 'Blues'
scatterprops = { 'lw' : 0, 's' : 8., 'marker' : 'o' }

pprops = { 'xlim'      : [ 1., 8.],
           'xticks'    : [ 2., 4., 6., 8.],
           'xticklabels' : [r'$-2$', r'$-4$', r'$-6$', r'$-8$'],
           'xaxstart'  : 1,
           'ylim'      : [ 0., 1.],
           'yticks'    : [ 0., 0.25, 0.50, 0.75, 1.00],
           'xlabel'    : 'Median binding free energy',
           'ylabel'    : r'$Q$',
           'plotprops' : contourprops,
           'zorder'    : 10 }

mp.plot(type='contour', ax=axLL[0], x=[xx], y=[yy], z=[zz], **pprops)
axLL[0].text(6.5, 0.80, 'PGT121-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})
axLL[0].text(5.5, 0.25, '3BNC60-like', size=sizelabel, color='k', ha='left', va='center',
                        bbox={'facecolor':'w', 'alpha':0.5, 'pad':2, 'lw':0})
    
# Schematic

hiSchem  = mpimg.imread('figures/hiEc.png')
loSchem  = mpimg.imread('figures/loEc.png')
keySchem = mpimg.imread('figures/legend2.png')
axS[1].imshow(hiSchem)
axS[2].imshow(loSchem)
axKey.imshow(keySchem, zorder=-200)
for ax in [axS[1], axS[2], axKey]:
    for a in ['left', 'right', 'bottom', 'top']: 
        ax.spines[a].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
axS[2].text( 0.02, LoBox[3]-0.01, 'B', transform=fig.transFigure, **def_sublabelprops)
axS[1].text( 0.02, HiBox[3]-0.01, 'A', transform=fig.transFigure, **def_sublabelprops)
axLL[0].text(0.02, LoBox[2]-0.08, 'C', transform=fig.transFigure, **def_sublabelprops)
axS[1].text(0.07, (HiBox[2]+HiBox[3])/2.-0.03, 'Strong initial ' + r'$E_c$', ha='center', va='center',
            transform=fig.transFigure, **def_labelprops)
axS[2].text(0.07, (LoBox[2]+LoBox[3])/2.-0.03, 'Weak initial ' + r'$E_c$', ha='center', va='center', 
            transform=fig.transFigure, **def_labelprops)
tprops['size'] = sizelabel
axKey.text(0.080, 0.965,        'viral\nspike', transform=fig.transFigure, ha='center', va='center', **tprops)
axKey.text(0.175, 0.965,   'conserved\nregion', transform=fig.transFigure, ha='center', va='center', **tprops)
axKey.text(0.290, 0.965, 'antibody\nfootprint', transform=fig.transFigure, ha='center', va='center', **tprops)

# Background

cBG   = '#F5F5F5'
dy    = 0.07
bgHi  = axS[1].axis()
recHi = matplotlib.patches.Rectangle(xy=(0,HiBox[2]-0.8*dy), width=1., height=HiBox[3]-HiBox[2]+(1.*dy), 
                                     transform=fig.transFigure, ec=None, fc=cBG, clip_on=False, zorder=-100)
recHi = axS[1].add_patch(recHi)

bgLo  = axS[2].axis()
recLo = matplotlib.patches.Rectangle(xy=(0,LoBox[2]-0.8*dy), width=1., height=LoBox[3]-LoBox[2]+(1.*dy), 
                                     transform=fig.transFigure, ec=None, fc=cBG, clip_on=False, zorder=-100)
recLo = axS[2].add_patch(recLo)
    
# Save data
plot.savefig('figures/figure-S5-contour-breadth-Ec-Q.pdf', dpi = 1000, 
              facecolor = fig.get_facecolor(), edgecolor=None, **figprops)
plot.close(fig)

print('Done.')

Plotting densities...
Plotting breadth...
Plotting Q vs breadth...
Done.


# 3. Parameters for affinity maturation simulations

`am.py` uses global parameters set at the start of the function for simulations. The parameters that we used in the main cases considered in the paper,

1. single antigen,
2. high/low $E_c$ (standard), and 
3. high/low $E_c$ (low overlap, $\lambda=0.8$)

are given below. Any parameters not listed below are the same for all data sets and match with those currently in `am.py`.

### Parameters for single antigen case

In [ ]:
p_var  = 0.10        # probability that a CDR mutation affects the variable region
p_cons = 1.0 - p_var # probability that a CDR mutation affects the constant region

Ec_start = 0.        # starting value for Ec
nb_Ag    = 1         # number of antigens
conc     = 1.11      # antigen concentration

### Parameters for standard case with multiple antigens

In [ ]:
# High Ec

p_var  = 0.10        # probability that a CDR mutation affects the variable region
p_cons = 1.0 - p_var # probability that a CDR mutation affects the constant region

Ec_start = 4.        # starting value for Ec
nb_Ag    = 10        # number of antigens
conc     = 0.98      # antigen concentration

# Low Ec

p_var  = 0.10        # probability that a CDR mutation affects the variable region
p_cons = 1.0 - p_var # probability that a CDR mutation affects the constant region

Ec_start = 0.        # starting value for Ec
nb_Ag    = 10        # number of antigens
conc     = 1.18      # antigen concentration

### Parameters for low overlap ($\lambda=0.8$)

In [ ]:
# High Ec

p_var  = 0.20        # probability that a CDR mutation affects the variable region
p_cons = 1.0 - p_var # probability that a CDR mutation affects the constant region

Ec_start = 4.        # starting value for Ec
nb_Ag    = 5         # number of antigens
conc     = 1.08      # antigen concentration

# Low Ec

p_var  = 0.20        # probability that a CDR mutation affects the variable region
p_cons = 1.0 - p_var # probability that a CDR mutation affects the constant region

Ec_start = 0.        # starting value for Ec
nb_Ag    = 5         # number of antigens
conc     = 1.21      # antigen concentration